In [2]:
from functools import partial
from geopy.geocoders import Nominatim
import WazeRouteCalculator
geolocator = Nominatim(user_agent="useer_agent", domain='nominatim.openstreetmap.org')
geocode = partial(geolocator.geocode, language="it", country_codes = "ita")


In [6]:
from pandas_ods_reader import read_ods
import pandas as pd
path = "IndirizziBiblio_21-10-2019.ods"
df = read_ods(path, 'Questionari_normalizza')
df=df[df['Calendarizzata (sì/no)']=='sì']
df=df.astype({'Cap':'int32'})
df.head()

,Denominazione,Comune,Provincia,Indirizzo,Cap,Calendarizzata (sì/no)
0,Agnadello - Biblioteca Comunale,Agnadello,CR,"via Marconi, 6",26020,sì
1,Annicco - Biblioteca Comunale Remo Contardi,Annicco,CR,"piazza Giuseppe Garibaldi, 6",26021,sì
3,Bagnolo Cremasco - Biblioteca Comunale,Bagnolo Cremasco,CR,"piazza Roma, 3",26010,sì
4,Bonemerse - Biblioteca Comunale,Bonemerse,CR,"via Roma, 25",26040,sì
7,Camisano - Biblioteca Comunale,Camisano,CR,"via Trieste, 57",26010,sì


In [8]:
def getLocation(name):
    geolocator = Nominatim(user_agent="useer_agent")
    geocode = partial(geolocator.geocode, language="it")
    location =geolocator.geocode(name)
    return location.latitude, location.longitude
#print(lat, lon)

In [9]:
def getName(row):
    name=row.Comune + ','
    try:
        name=name + row.Indirizzo[row.Indirizzo.lower().index('via')+3:len(row.Indirizzo)]
    except:
        try:
            name=name + row.Indirizzo[row.Indirizzo.lower().index('piazza')+6:len(row.Indirizzo)]
        except:
            try:
                name=name + row.Indirizzo[row.Indirizzo.lower().index('piazzetta')+9:len(row.Indirizzo)]
            except:
                name=name + row.Indirizzo[row.Indirizzo.lower().index('vicolo')+6:len(row.Indirizzo)]
           
    return name

In [10]:
from tqdm.notebook import tqdm as tqdm

In [51]:
import csv
x=[]
y=[""]
for i in range(len(df)):
    x.append(getName(df.iloc[i]))
x.append("Cremona, Sesto 39")

for i in range(len(df)):
    y.append(getName(df.iloc[i]))
y.append("Ostiano")
y.append("San Giovanni in Croce")

with open('times.csv', mode='a') as time_file:
        time_file = csv.writer(time_file, delimiter=';')
        time_file.writerow(y)

with open('distances.csv', mode='a') as distance_file:
    distance_file = csv.writer(distance_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    distance_file.writerow(y)

y.pop(0)


''

In [ ]:
import time
dataf=pd.read_csv("times.csv", encoding = "ISO-8859-1", delimiter=";", index_col=0, error_bad_lines=False)
start=len(dataf)
#while start<len(x):
try:
    dataf=pd.read_csv("times.csv", encoding = "ISO-8859-1", delimiter=";", index_col=0, error_bad_lines=False)
    start=len(dataf)
    #Outer loop over cities (from)
    for j in range(start, len(x)):
        lat, lon=getLocation(x[j])
        from_address=str(lat)+", "+str(lon)
        double_error = False
        t=[x[j]]
        d=[x[j]]
        #Inner loop over cities (to)
        for i in tqdm(range(len(y)), desc=str(j)):
            wait=2
            #Loop until get distance
            while True:
                region = 'EU'
                lat, lon = getLocation(y[i])
                to_address=str(lat)+", "+str(lon)
                try:
                    route = WazeRouteCalculator.WazeRouteCalculator(from_address, to_address, region)
                    route_time, route_distance = route.calc_route_info(real_time=False)
                except:
                    #On error wait (to avoid out of request) and retry to get route
                    #print(x[j]+"\n"+y[i])
                    time.sleep(wait)
                    wait=wait+2
                    continue
                t.append(route_time)
                d.append(route_distance)
                break
        with open('times.csv', mode='a') as time_file:
            time_file = csv.writer(time_file, delimiter=';', quoting=csv.QUOTE_MINIMAL)
            time_file.writerow(t)

        with open('distances.csv', mode='a') as distance_file:
            distance_file = csv.writer(distance_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            distance_file.writerow(d)
except:
    time.sleep(5)
    pass


In [14]:
dataf=pd.read_csv("times.csv", encoding = "ISO-8859-1", delimiter=";", index_col=0, error_bad_lines=False)
dataf

,"Agnadello, Marconi, 6","Annicco, Giuseppe Garibaldi, 6","Bagnolo Cremasco, Roma, 3","Bonemerse, Roma, 25","Camisano, Trieste, 57","Capergnanica, XI febbraio, 7","Capralba, Piave, 2","Casalbuttano ed Uniti, Jacini, 23","Casale Cremasco-Vidolasco, Roma, 56","Casalmaggiore, Guglielmo Marconi, 8",...,"Trescore Cremasco, Vittorio Veneto, 1","Trigolo, Roma, 26","Vaiano Cremasco, Gloriosi Caduti, 3","Vailate, Dante Alighieri, 7","Vescovato, Roma, 44","Cremona, Felice Cavallotti, 2","Cremona, Palosca n. 2","Cremona, Palosca n. 2.1",Ostiano,San Giovanni
"Agnadello, Marconi, 6",0.18333333333333332,41.266.666.666.666.600,17.033.333.333.333.300,5.853.333.333.333.330,22.883.333.333.333.300,17.00,10.333.333.333.333.300,4.606.666.666.666.660,21.733.333.333.333.300,87.65,...,12.55,32.95,12.65,7.166.666.666.666.660,6.178.333.333.333.330,5.518.333.333.333.330,5.606.666.666.666.660,5.606.666.666.666.660,66.7,66.7
"Annicco, Giuseppe Garibaldi, 6",3.996.666.666.666.660,0.0,33.1,25.483.333.333.333.300,33.65,28.20,38.266.666.666.666.600,10.55,3.261.666.666.666.660,5.471.666.666.666.660,...,3.571.666.666.666.660,15.266.666.666.666.600,32.85,43.05,28.85,22.333.333.333.333.300,23.05,23.05,37.766.666.666.666.600,37.766.666.666.666.600
"Bagnolo Cremasco, Roma, 3",15.15,32.266.666.666.666.600,0.0,49.483.333.333.333.300,22.9,7.55,19.583.333.333.333.300,40.35,21.866.666.666.666.600,78.55,...,12.116.666.666.666.600,24.75,5.983.333.333.333.330,21.583.333.333.333.300,52.8,46.166.666.666.666.600,47.083.333.333.333.300,47.083.333.333.333.300,61.85,61.85
